In [280]:
import pickle
with open('data/data_batch_1', 'rb') as fo:
    image_data = pickle.load(fo, encoding='bytes')
n_samples = 100
sampled_data = image_data[b'data'][:1000]
sampled_labels = image_data[b'labels'][:1000]
#data
train_data = sampled_data[n_samples:]
test_data = sampled_data[:n_samples]
test_labels = sampled_labels[:n_samples]
train_labels = sampled_labels[n_samples:]

#single test instance
test_one = test_data[0]
test_one = test_one.reshape((1, 3072))
test_label_one = test_labels[0]


[6, 9, 9, 4, 1, 1, 2, 7, 8, 3, 4, 7, 7, 2, 9, 9, 9, 3, 2, 6, 4, 3, 6, 6, 2, 6, 3, 5, 4, 0, 0, 9, 1, 3, 4, 0, 3, 7, 3, 3, 5, 2, 2, 7, 1, 1, 1, 2, 2, 0, 9, 5, 7, 9, 2, 2, 5, 2, 4, 3, 1, 1, 8, 2, 1, 1, 4, 9, 7, 8, 5, 9, 6, 7, 3, 1, 9, 0, 3, 1, 3, 5, 4, 5, 7, 7, 4, 7, 9, 4, 2, 3, 8, 0, 1, 6, 1, 1, 4, 1]


In [194]:
def calculate_distance(a, b):
    # one row [3072] length is one image pixels of 1024 red, 
    # 1024 green and 1024 blue pixels 
    # distance between two images ?
    #a= np.array([1,2,3,4]) 
    #b = np.array([5,6,7,8])
    distance = np.sqrt(np.sum((a-b)**2))
    dist = np.linalg.norm(a-b)
    return distance   
#calculate_distance()

In [195]:
def find_neighbors(k, train_pca, test_pca):
#     samp_train = np.array([[2, 2], [4, 4], [3, 3], [4.5,4.5], [4.9,4.9]])
#     samp_labels = ['cats', 'fishes', 'birds', 'fishes', 'dogs']
#     samp_test = np.array([5, 5])
#     samp_test_labels = ['dogs']
    distance = []
    neighbors = []
#    k = 3
    for i in range(len(train_pca)):
        distance.append((train_pca[i],calculate_distance(train_pca[i], test_pca), train_labels[i]))
    distance.sort(key=lambda x: x[1])
    for x in range(k):
        neighbors.append(distance[x])
    return neighbors
#find_neighbors()

In [233]:
def find_class(neighbors):
    vote_dict = {}
    #neighbors = find_neighbors()
    for neighbor in neighbors:
        label = neighbor[-1]
        distance = neighbor[1]
        if label in vote_dict:
            #count
            vote_dict[label] += 1/distance
        else:
            vote_dict[label] = 1/distance
    #print(vote_dict)
    return max(vote_dict.items(), key=lambda x: x[1])[0]
#find_class()         

In [279]:
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
def knn(k, d, n):
    # PCA
    train_grayed = convert_gray(train_data, 1000-n, 1024)
    test_grayed = convert_gray(test_data, n, 1024)
    pca_obj, train_pca = do_pca(train_grayed, d)
    test_pca = reduce_dimension(test_grayed, pca_obj)
    # knn
    labels = []
    accuracy=0
    for i in range(len(test_pca)):
        neighbors = find_neighbors(k, train_pca, test_pca[i])
        prediction = find_class(neighbors)
        actual = test_labels[i]
        if prediction == actual:
            accuracy+=1
        labels.append((prediction,actual))
    print(labels)
    print(accuracy)
knn(3, 400, 100)

[(6, 6), (8, 9), (9, 9), (0, 4), (2, 1), (1, 1), (8, 2), (0, 7), (0, 8), (0, 3), (4, 4), (8, 7), (8, 7), (4, 2), (4, 9), (8, 9), (0, 9), (3, 3), (7, 2), (6, 6), (0, 4), (6, 3), (6, 6), (4, 6), (0, 2), (7, 6), (6, 3), (6, 5), (4, 4), (8, 0), (4, 0), (8, 9), (4, 1), (6, 3), (8, 4), (2, 0), (6, 3), (4, 7), (0, 3), (3, 3), (2, 5), (2, 2), (2, 2), (8, 7), (0, 1), (2, 1), (8, 1), (4, 2), (2, 2), (0, 0), (6, 9), (2, 5), (2, 7), (0, 9), (0, 2), (2, 2), (4, 5), (0, 2), (4, 4), (3, 3), (0, 1), (4, 1), (2, 8), (2, 2), (0, 1), (5, 1), (4, 4), (8, 9), (0, 7), (8, 8), (4, 5), (3, 9), (9, 6), (1, 7), (2, 3), (4, 1), (8, 9), (5, 0), (4, 3), (0, 1), (3, 3), (6, 5), (4, 4), (4, 5), (5, 7), (6, 7), (6, 4), (7, 7), (0, 9), (8, 4), (2, 2), (6, 3), (0, 8), (0, 0), (8, 1), (0, 6), (3, 1), (2, 1), (0, 4), (6, 1)]
24


In [225]:
def convert_gray(data, row, column):
    img = data[:]
    grayed = np.zeros((row, column))

#     print(img.dtype, img.max())
#     img = np.reshape(img,(3,32,32))
#     img = np.transpose(img, (1,2,0))
#     print(img.dtype, img.max())
    for i in range(len(img)):
        grayed[i] = img[i][:1024]*0.299 + img[i][1024:2048]*0.587 + img[i][2048:]*0.114
#     reshape = np.reshape(grayed[0],(32,32))
#     plt.imshow(reshape, 'gray')
#     plt.show()
    return grayed

In [235]:
def do_pca(gray_data, d):
    
    pca = PCA(n_components=d, svd_solver='full')
    pca_obj = pca.fit(gray_data)
    train_pca = pca_obj.transform(gray_data)
    #print(train_pca.shape)
    return (pca_obj, train_pca)

In [227]:
def reduce_dimension(gray_data, pca):
    return pca.transform(gray_data)
    